# Práctico 7: Generadores centrales de patrones (CPGs)

Un generador central de patrones es un circuito neuronal que puede producir, de forma autónoma, un patrón de actividad, como una oscilación, que puede activar los músculos de manera confiable. Aunque una entrada externa puede encender o apagar el generador central de patrones, o cambiarlo entre modos, y la neuromodulación puede ajustar características del patrón, el circuito neuronal de un generador central de patrones no depende de ninguna entrada externa para producir el patrón de actividad. Cuando los animales muestran contracciones musculares coordinadas estereotipadas que acompañan un comportamiento (como la respiración, la locomoción o la masticación), los generadores centrales de patrones suelen estar involucrados.

## Configuración

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Funciones utilitarias

In [ ]:
def I_inj(t, amp):
  """Crea una función de corriente inyectada con un pulsos cuadrado.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    amp (float): amplitud del pulso, en uA/cm^2.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio = inicios[1]
  fin = inicios[6]
  
  def I(t):
    return amp * (t > inicio) - amp * (t > fin)

  return I

def I_inj_ruido(t, amp, media=0, desvio=1):
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio = inicios[1]
  fin = inicios[6]

  base = amp * (t > inicio) - amp * (t > fin)

  vals = base + np.random.normal(media, desvio, size=t.shape)

  def I(tq):
      return np.interp(tq, t, vals)

  return I

def vdot(v, u, I):
  return (0.04 * v + 5) * v + 140 - u + I

def udot(u, v, a, b):
  return a * (b * v - u)

def step_izhikevich(v, u, I, a, b, c, d, dt):
  spiked = False
  if v < threshold:
    v = v + dt * vdot(v, u, I)
    u = u + dt * udot(u, v, a, b)
  else:
    spiked = True
    v = c
    u = u + d
  return v, u, spiked

### Funciones de graficado

In [ ]:
def plot_izikhevich(t, I1, I2, v1, u1, v2, u2, I_syn1, I_syn2):
    # Graficamos
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 6), gridspec_kw={"height_ratios": [3, 2, 1]})
    
    ax1.plot(t, v1, 'r')
    ax1.plot(t, v2, 'g')

    ax2.plot(t, u1, 'r')
    ax2.plot(t, u2, 'g')

    #ax3.plot(t, I(t))
    ax3.plot(t, I1(t), 'r')
    ax3.plot(t, I2(t), 'g')
    ax3.plot(t, I_syn1, 'r')
    ax3.plot(t, I_syn2, 'g')
    ax3.set_ylim(-11, 11)  
    ax3.set_ylabel("I (mA)")
    ax3.set_xlabel("t (ms)")

    plt.show()

def neuron_params_widgets(label_prefix, defaults=(0.02, 0.2, -55, 4)):
    a, b, c, d = defaults
    return widgets.VBox([
        widgets.HTML(f"<b>Parámetros de la {label_prefix}</b>"),
        widgets.FloatSlider(value=a, min=0, max=0.2, step=0.01, description='a'),
        widgets.FloatSlider(value=b, min=0, max=0.3, step=0.01, description='b'),
        widgets.FloatSlider(value=c, min=-75, max=-50, step=0.1, description='c'),
        widgets.FloatSlider(value=d, min=0, max=10, step=0.1, description='d')
    ])

def global_params_widgets():
    return widgets.VBox([
        widgets.HTML(f"<b>Parámetros globales</b>"),
        widgets.FloatSlider(value=10, min=-10, max=10, step=0.1, description='I_iny (mA)'), 
        widgets.FloatSlider(value=5, min=0.1, max=30, step=0.1, description='Syn. dur.'), 
        widgets.FloatSlider(value=-5, min=-10, max=10, step=0.1, description='I_syn_amp')
    ])

## Modelo de Izhikevich

Izhikevich (2003) propuso un modelo simple de neurona que se describe con dos ecuaciones diferenciales acopladas. Una para el potencial de membrana $v$ y otra para el valor de recobro $u$:

$$\begin{split}
\dfrac{dv(t)}{dt} &= 0.04 v^2(t) + 5 v(t) + 140 - u + I(t) \\
\dfrac{du(t)}{dt} &= a(bv - u)
\end{split}$$

En conjunto con las condiciones de reestablecimiento:

$$\begin{split}
v(v > \theta) &= c \\
u(v > \theta) &= u + d
\end{split}$$

Este modelo cumple con el requisito de ser computacionalmente eficiente y además de producir un variado abanico de patrones de disparos observados en neuronas corticales.

### Rol de los parámetros en el modelo de Izhikevich

El modelo de Izhikevich utiliza cuatro parámetros principales (a, b, c, d) para modelar una amplia variedad de comportamientos neuronales.

Aquí se describe el rol de cada parámetro:

* **`a`**: Controla la **velocidad de recuperación** de la variable de adaptación `u`. Valores bajos de `a`  hacen que la neurona se recupere lentamente después de un pico, mientras que valores altos producen una recuperación rápida. Esto influye en la frecuencia de disparo de la neurona.

* **`b`**:  Describe la **sensibilidad** de la variable de adaptación `u` al potencial de membrana `v`.  Valores altos de `b` hacen que la neurona sea más sensible a los cambios en `v`, lo que puede producir comportamientos como el rebote post-inhibitorio.

* **`c`**: Representa el **potencial de reinicio** de la membrana después de un pico.  Este valor determina a qué voltaje se restablece la membrana después de que la neurona ha disparado.

* **`d`**:  Controla la **recuperación de la variable de adaptación `u`** después de un pico.  Un valor alto de `d`  provoca un aumento rápido en `u` después de un pico, lo que puede influir en la duración del período refractario y la frecuencia de disparo.

En resumen, los parámetros `a`, `b`, `c` y `d`  permiten ajustar la dinámica del modelo de Izhikevich para reproducir una amplia gama de comportamientos neuronales, como disparos tónicos, ráfagas, rebotes post-inhibitorios, y más.

Ajustar estos parámetros de manera precisa es crucial para simular neuronas específicas y para estudiar la dinámica de las redes neuronales.

## Oscilador de medio centro

El oscilador de medio centro se logra con dos neuronas, cada una de las cuales inhibe a la otra. La actividad de cada neurona oscila en antifase con la otra, de modo que pueden producir contracciones musculares que alternan entre dos músculos antagonistas. Estas contracciones alternas entre los dos lados del cuerpo de un animal constituyen la unidad básica para la locomoción mediante deslizamiento o serpenteo, lo cual es común en animales largos y sin patas (como lampreas, anguilas, serpientes, gusanos, etc.).

Comenzamos definiendo dos constantes:

In [ ]:
v0 = -75        # mV
threshold = 30  # mV

Y la función de integración. Lo importante acá es notar como en cada paso, se inyecta corriente en cada neurona más la corriente generada por la otra neurona.

In [ ]:
def integrate_izikhevich(t, I1, I2, I_syn_amp, syn_duration, a1=0.02, b1=0.2, c1=-65, d1=2, a2=0.02, b2=0.2, c2=-65, d2=2):
  # Variable de voltaje
  v1 = np.zeros_like(t)
  v2 = np.zeros_like(t)
  v1[0] = v2[0] = v0

  # Variable de recobro
  u1 = np.zeros_like(t)
  u2 = np.zeros_like(t)
  u1[0] = v1[0] * b1
  u2[0] = v2[0] * b2
  
  # Calculamos el valor de salto en el tiempo
  dt = t[1] - t[0]

  I_syn1 = np.zeros_like(t)
  I_syn2 = np.zeros_like(t)

  decay = np.exp(-dt / syn_duration)
  
  # Integramos con el método de Euler
  for step in range(len(t)-1):
    v1[step+1], u1[step+1], spike1 = step_izhikevich(v1[step], u1[step], I1(t[step]) + I_syn2[step], a1, b1, c1, d1, dt)
    v2[step+1], u2[step+1], spike2 = step_izhikevich(v2[step], u2[step], I2(t[step]) + I_syn1[step], a2, b2, c2, d2, dt)

    if spike1:
      I_syn1[step] += I_syn_amp
    if spike2:
      I_syn2[step] += I_syn_amp

    I_syn1[step + 1] = I_syn1[step] * decay
    I_syn2[step + 1] = I_syn2[step] * decay
      
  return v1, u1, v2, u2, I_syn1, I_syn2

In [ ]:
neuron1_box = neuron_params_widgets("neurona 1")
neuron2_box = neuron_params_widgets("neurona 2")
global_controls = global_params_widgets()
controls = {
    'amp': global_controls.children[1],
    'syn_duration': global_controls.children[2],
    'I_syn_amp': global_controls.children[3],
    'a1': neuron1_box.children[1],
    'b1': neuron1_box.children[2],
    'c1': neuron1_box.children[3],
    'd1': neuron1_box.children[4],
    'a2': neuron2_box.children[1],
    'b2': neuron2_box.children[2],
    'c2': neuron2_box.children[3],
    'd2': neuron2_box.children[4]
}

def simulate(amp, syn_duration, I_syn_amp, a1, b1, c1, d1, a2, b2, c2, d2):
    t = np.arange(0, 400, 0.05)
    I1 = I_inj_ruido(t, amp)
    I2 = I_inj_ruido(t, amp)
    V1, u1, V2, u2, I_syn1, I_syn2 = integrate_izikhevich(
        t, I1, I2, I_syn_amp, syn_duration,
        a1, b1, c1, d1,
        a2, b2, c2, d2
    )
    plot_izikhevich(t, I1, I2, V1, u1, V2, u2, I_syn1, I_syn2)

display(widgets.HBox([widgets.VBox([global_controls, neuron1_box, neuron2_box]), widgets.interactive_output(simulate, controls)]))

## Referencias

Izhikevich, E. M. (2003). Simple model of spiking neurons. *IEEE Transactions on Neural Networks*, 14(6), 1569–1572. [doi:10.1109/tnn.2003.820440](https://doi.org/10.1109/TNN.2003.820440)